# Projeto 3: Robô BTC

* Desafio: Como gerar sinais de compra e venda no Bitcoin?

### Passo a passo:

   **Passo 1** - Importar bibliotecas
   
   **Passo 2** - Pegar dados de cotação do Yahoo Finance
   
   **Passo 3** - Preparar e calcular dados importantes pro modelo
   
   **Passo 4** - Gerar sinais de compra
   
   **Passo 5** - Gerar sinais de venda

   **Passo 6** - Gerar operações de compra e venda históricas
   
   **Passo 7** - Criar um ID para todos os trades históricos na tabela
   
   **Passo 8** - Calcular retornos do modelo



   
   <br>
   
   
-------------------

# Passo 1 - Importar as bibliotecas

In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import mplcyberpunk
plt.style.use("cyberpunk")

# Passo 2 - Pegar dados de cotação do Yahoo Finance

In [ ]:
ticker = "BTC-USD"

In [ ]:
dados = yf.download(ticker, auto_adjust = False)
dados = dados.droplevel(1, axis = 1)
dados

# Passo 3 - Preparar e calcular dados importantes pro modelo

In [ ]:
dados["retorno"] = dados["Adj Close"].pct_change()
dados_retornos_completos = dados["retorno"]

In [ ]:
dados["media_maximas"] = dados["High"].rolling(window = 20).mean()
dados["media_minimas"] = dados["Low"].rolling(window = 20).mean()

# Passo 4 - Gerar sinais de compra

In [ ]:
dados["sinal_compra"] = 0
dados["sinal_compra"] = (dados["Close"] > dados["media_maximas"]).astype(int)

# Passo 5 - Gerar sinais de venda

In [ ]:
dados["sinal_venda"] = 0
dados["sinal_venda"] = (dados["Close"] < dados["media_minimas"]).astype(int)

# Passo 6 - Gerar operações

In [ ]:
# Gerar posição: Compra no sinal de compra e zera no sinal de venda

dados["posicao"] = 0

for i in range(1, len(dados)):

    if dados['sinal_compra'].iloc[i] == 1:

        dados["posicao"].iloc[i] = 1

    elif dados['sinal_venda'].iloc[i] == 1:

        dados["posicao"].iloc[i] = 0

    else:

        if (dados['posicao'].iloc[i-1] == 1) and (dados['sinal_venda'].iloc[i] == 0):

            dados["posicao"].iloc[i] = 1
        
        else:

            dados["posicao"].iloc[i] = 0

In [ ]:
dados["posicao"] = dados["posicao"].shift()

# Passo 7 - Criar um ID para todos os trades históricos na tabela

* Nosso objetivo é conseguir agrupar todos os trades através de um ID

In [ ]:
dados['trades'] = (dados['posicao'] != dados['posicao'].shift()).cumsum() 

dados['trades'] = dados['trades'].where(dados['posicao'] == 1) #só interessado nos IDs quando estávamos comprados 

dados = dados.dropna(subset = "trades")

# Passo 8 - Calcular retornos de todos os trades

In [ ]:
df_retorno_acumulado = (1 + dados["retorno"]).cumprod() - 1
dados_retornos_completos_acum = (1 + dados_retornos_completos).cumprod() - 1
dados_retornos_completos_acum.plot(label = "Ativo")
df_retorno_acumulado.plot(label = "Modelo")
plt.legend()

In [ ]:
print(df_retorno_acumulado.iloc[-1], dados_retornos_completos_acum.iloc[-1])